In [2]:
df = pd.read_csv("input/train.csv", index_col=False)

In [3]:
df.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014-01-29,69,38.0,7.0,10.0,1.0,2001.0,2.0,11.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014-04-07,55,NaN,2.0,1.0,4.0,NaN,2.0,10.0,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012-05-18,30,16.0,2.0,NaN,NaN,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013-02-08,44,43.0,1.0,NaN,NaN,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014-01-10,45,28.0,3.0,5.0,2.0,1960.0,2.0,5.0,...,20,2,0,4,16,1,4,47,5,6700000


In [4]:
df.shape

(24376, 292)

# Preprocessing

## Check types

In [5]:
df.dtypes.value_counts()

int64      157
float64    119
object      16
dtype: int64

## Get notnums w/ indices

In [6]:
notnum = df.dtypes[df.dtypes == type(object)]
notnum

f1      object
f11     object
f12     object
f29     object
f33     object
f34     object
f35     object
f36     object
f37     object
f38     object
f39     object
f40     object
f106    object
f114    object
f118    object
f152    object
dtype: object

In [7]:
notnum_index = list(notnum.index)
print(notnum_index)

['f1', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [8]:
df[notnum_index].head()

,f1,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014-01-29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014-04-07,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012-05-18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013-02-08,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014-01-10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f1

In [9]:
ymd = df['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'f1y', 1: 'f1m', 2: 'f1d'})
ymd

,f1y,f1m,f1d
0,2014,1,29
1,2014,4,7
2,2012,5,18
3,2013,2,8
4,2014,1,10
...,...,...,...
24371,2014,12,21
24372,2012,5,10
24373,2013,12,18
24374,2014,12,8


In [10]:
df_f1 = df.drop(['f1'], axis=1)
df_f1.insert(1, 'f1y', ymd['f1y'])
df_f1.insert(2, 'f1m', ymd['f1m'])
df_f1.insert(3, 'f1d', ymd['f1d'])

In [11]:
notnum_index = ['f1y', 'f1m', 'f1d'] + notnum_index[1:]
print(notnum_index)

['f1y', 'f1m', 'f1d', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [12]:
df_f1[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f11

In [13]:
f11_unique = df_f1['f11'].unique()
f11_unique

array(['Investment', 'OwnerOccupier'], dtype=object)

In [14]:
f11_encode = dict(zip(f11_unique, range(len(f11_unique))))
f11_encode

{'Investment': 0, 'OwnerOccupier': 1}

In [15]:
df_f11 = df_f1.replace({'f11': f11_encode})
df_f11[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f12

In [16]:
f12_unique = df_f11['f12'].unique()
f12_unique

array(['Juzhnoe Butovo', 'Poselenie Filimonkovskoe', 'Lomonosovskoe',
       'Juzhnoe Tushino', 'Ochakovo-Matveevskoe', 'Poselenie Sosenskoe',
       'Orehovo-Borisovo Juzhnoe', 'Hovrino', 'Sokol',
       'Nagatinskij Zaton', "Kon'kovo", 'Obruchevskoe',
       'Poselenie Vnukovskoe', 'Preobrazhenskoe', 'Poselenie Moskovskij',
       'Bibirevo', 'Jasenevo', 'Mitino', 'Birjulevo Vostochnoe',
       'Koptevo', 'Beskudnikovskoe', 'Butyrskoe', 'Losinoostrovskoe',
       'Novo-Peredelkino', 'Caricyno', 'Veshnjaki', 'Tverskoe', 'Perovo',
       'Jaroslavskoe', 'Strogino', 'Nekrasovka', 'Horoshevo-Mnevniki',
       'Filevskij Park', 'Bogorodskoe', 'Savelki',
       'Poselenie Desjonovskoe', "Chertanovo Central'noe",
       'Timirjazevskoe', 'Zapadnoe Degunino', 'Pokrovskoe Streshnevo',
       'Severnoe Butovo', 'Kapotnja', 'Sviblovo', 'Danilovskoe',
       'Sokolinaja Gora', 'Vostochnoe Izmajlovo', 'Matushkino',
       'Krjukovo', "Krasnosel'skoe", 'Ajeroport', 'Taganskoe',
       'Cheremushki

In [17]:
f12_encode = dict(zip(f12_unique, range(len(f12_unique))))
f12_encode

{'Juzhnoe Butovo': 0,
 'Poselenie Filimonkovskoe': 1,
 'Lomonosovskoe': 2,
 'Juzhnoe Tushino': 3,
 'Ochakovo-Matveevskoe': 4,
 'Poselenie Sosenskoe': 5,
 'Orehovo-Borisovo Juzhnoe': 6,
 'Hovrino': 7,
 'Sokol': 8,
 'Nagatinskij Zaton': 9,
 "Kon'kovo": 10,
 'Obruchevskoe': 11,
 'Poselenie Vnukovskoe': 12,
 'Preobrazhenskoe': 13,
 'Poselenie Moskovskij': 14,
 'Bibirevo': 15,
 'Jasenevo': 16,
 'Mitino': 17,
 'Birjulevo Vostochnoe': 18,
 'Koptevo': 19,
 'Beskudnikovskoe': 20,
 'Butyrskoe': 21,
 'Losinoostrovskoe': 22,
 'Novo-Peredelkino': 23,
 'Caricyno': 24,
 'Veshnjaki': 25,
 'Tverskoe': 26,
 'Perovo': 27,
 'Jaroslavskoe': 28,
 'Strogino': 29,
 'Nekrasovka': 30,
 'Horoshevo-Mnevniki': 31,
 'Filevskij Park': 32,
 'Bogorodskoe': 33,
 'Savelki': 34,
 'Poselenie Desjonovskoe': 35,
 "Chertanovo Central'noe": 36,
 'Timirjazevskoe': 37,
 'Zapadnoe Degunino': 38,
 'Pokrovskoe Streshnevo': 39,
 'Severnoe Butovo': 40,
 'Kapotnja': 41,
 'Sviblovo': 42,
 'Danilovskoe': 43,
 'Sokolinaja Gora': 44,
 'V

In [18]:
df_f12 = df_f11.replace({'f12': f12_encode})
df_f12[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,1,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,2,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,3,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,4,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f29 - f118

In [19]:
notnum_bools = notnum_index[5:-1]
print(notnum_bools)

['f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118']


In [20]:
bools_unique = set(np.array([df_f12[col].unique() for col in notnum_bools]).flatten())
bools_unique

{'no', 'yes'}

In [21]:
bools_encode = dict(zip(bools_unique, range(len(bools_unique))))
bools_encode

{'yes': 0, 'no': 1}

In [22]:
df_bools = df_f12.replace(dict.fromkeys(notnum_bools, bools_encode))
df_bools[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,good
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,no data
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,satisfactory
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,poor
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,satisfactory


## Process f152

In [23]:
f152_unique = df_bools['f152'].unique()
f152_unique

array(['good', 'no data', 'satisfactory', 'poor', 'excellent'],
      dtype=object)

In [24]:
f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
f152_encode

{'no data': 0, 'poor': 1, 'satisfactory': 2, 'good': 3, 'excellent': 4}

In [25]:
df_f152 = df_bools.replace({'f152': f152_encode})
df_f152[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,3
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,2
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,2


## Obtain result

In [26]:
df_final = df_f152.copy()
df_final.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


## Check NaNs

In [27]:
nans_cols = df_final.isna().sum()
nans_cols[nans_cols > 0].count()

51

In [28]:
df_final[nans_cols[nans_cols > 0].index].dtypes.unique()

array([dtype('float64')], dtype=object)

In [29]:
nans_rows = df_final.isna().sum(axis=1)
nans_rows[nans_rows >= 40]

172      40
554      40
785      40
1052     40
1359     40
         ..
22921    40
23466    40
23550    40
24236    40
24316    41
Length: 155, dtype: int64

In [30]:
print(list(nans_rows[nans_rows >= 40].index))

[172, 554, 785, 1052, 1359, 1495, 1646, 1812, 1997, 2133, 2249, 2302, 2529, 2542, 2678, 2684, 2781, 2801, 2861, 3251, 3254, 3699, 3904, 3959, 4208, 4365, 4544, 4656, 4834, 4929, 4941, 5151, 5159, 5277, 5474, 5621, 5634, 5691, 5805, 6092, 6191, 6256, 6347, 6388, 6524, 6862, 6876, 7066, 7202, 7259, 7506, 7508, 7575, 7618, 7625, 7694, 7796, 7980, 8119, 8184, 8484, 8673, 8834, 9290, 9632, 9693, 9786, 10247, 10427, 10572, 10639, 11016, 11281, 11394, 11648, 11857, 12249, 12556, 12625, 13322, 13330, 13498, 13592, 13696, 13772, 13911, 13996, 14014, 14062, 14331, 14345, 14518, 14925, 14999, 15026, 15086, 15313, 15359, 15493, 15679, 15812, 15916, 16036, 16129, 16532, 16820, 16965, 17116, 17534, 17574, 17971, 18011, 18037, 18053, 18256, 18311, 18334, 18429, 18475, 18532, 18630, 18632, 18691, 19155, 19187, 19253, 19429, 19523, 19792, 20051, 20087, 20279, 20435, 20650, 20927, 21346, 21410, 21508, 21570, 21735, 22029, 22096, 22195, 22210, 22235, 22258, 22324, 22356, 22477, 22608, 22921, 23466, 23550

## Remove too dirty entries

In [31]:
df_nans = df_final.drop(list(nans_rows[nans_rows >= 40].index))
df_nans.shape

(24221, 294)

## Impute missing values with KNN

In [32]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(weights='distance')

In [33]:
df_imp = imputer.fit_transform(df_nans)

In [34]:
df_imp2 = pd.DataFrame(df_imp, columns=df_final.columns)
df_imp2.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1.0,2014.0,1.0,29.0,69.0,38.00000,7.0,10.00000,1.000000,2001.000000,...,1.0,0.0,0.0,2.0,8.0,1.0,0.0,19.0,2.0,9500000.0
1,2.0,2014.0,4.0,7.0,55.0,48.13174,2.0,1.00000,4.000000,1591.150833,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,3837949.0
2,3.0,2012.0,5.0,18.0,30.0,16.00000,2.0,7.29370,3.732457,1961.927529,...,46.0,9.0,2.0,11.0,38.0,1.0,8.0,97.0,11.0,6250000.0
3,4.0,2013.0,2.0,8.0,44.0,43.00000,1.0,8.78533,1.428214,1970.661667,...,17.0,4.0,1.0,12.0,12.0,0.0,1.0,55.0,7.0,2000000.0
4,5.0,2014.0,1.0,10.0,45.0,28.00000,3.0,5.00000,2.000000,1960.000000,...,20.0,2.0,0.0,4.0,16.0,1.0,4.0,47.0,5.0,6700000.0


In [35]:
imputed = nans_cols[nans_cols > 0]
df_full = df_nans.copy()
df_full.reset_index(drop=True, inplace=True)
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


In [36]:
df_full[-3:]

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
24218,24374,2013,12,18,85,NaN,16.0,17.0,1.0,1.0,...,3,1,0,3,9,0,1,10,2,9609180
24219,24375,2014,12,8,47,27.0,2.0,10.0,2.0,1958.0,...,217,86,11,118,176,1,64,146,13,12900000
24220,24376,2015,4,30,85,60.0,10.0,22.0,4.0,2015.0,...,1,1,0,2,12,0,0,9,0,10371441


In [37]:
df_full[imputed.index] = df_imp2[imputed.index]
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.00000,7.0,10.00000,1.000000,2001.000000,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,48.13174,2.0,1.00000,4.000000,1591.150833,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.00000,2.0,7.29370,3.732457,1961.927529,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.00000,1.0,8.78533,1.428214,1970.661667,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.00000,3.0,5.00000,2.000000,1960.000000,...,20,2,0,4,16,1,4,47,5,6700000


In [38]:
True in df_full.isna().values

False

# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(3927)

# Input

In [3]:
df_train = pd.read_csv("input/train.csv", index_col=False)
df_test = pd.read_csv("input/test.csv", index_col=False)

# Preprocess

In [4]:
def process_f1(df_train, df_test, index):
    ymd_train = df_train['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'year', 1: 'month', 2: 'day'})
    ymd_test = df_test['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'year', 1: 'month', 2: 'day'})
    return pd.concat([df_train, ymd_train], axis=1).drop(['f1'], axis=1), \
           pd.concat([df_test, ymd_test], axis=1).drop(['f1'], axis=1), \
           index[1:]

In [5]:
def process_f11(df_train, df_test, index):
    f11_unique = df_train['f11'].unique()
    f11_encode = dict(zip(f11_unique, np.eye(len(f11_unique))))
    encode_train = pd.DataFrame([np.nan if k == np.nan else f11_encode[k] for k in df_train['f11']], columns=f11_unique)
    encode_test = pd.DataFrame([np.nan if k == np.nan else f11_encode[k] for k in df_test['f11']], columns=f11_unique)
    return pd.concat([df_train, encode_train], axis=1).drop(['f11'], axis=1), \
           pd.concat([df_test, encode_test], axis=1).drop(['f11'], axis=1), \
           index[1:]

In [6]:
def process_f12(df_train, df_test, index):
    f12_unique = df_train['f12'].unique()
    f12_encode = dict(zip(f12_unique, np.eye(len(f12_unique))))
    encode_train = pd.DataFrame([np.nan if k == np.nan else f12_encode[k] for k in df_train['f12']], columns=f12_unique)
    encode_test = pd.DataFrame([np.nan if k == np.nan else f12_encode[k] for k in df_test['f12']], columns=f12_unique)
    return pd.concat([df_train, encode_train], axis=1).drop(['f12'], axis=1), \
           pd.concat([df_test, encode_test], axis=1).drop(['f12'], axis=1), \
           index[1:]

In [7]:
def process_bool(df_train, df_test, index):
    bool_index = index[:-1]
    bool_encode = {'no': 0, 'yes': 1}
    encoding = dict.fromkeys(bool_index, bool_encode)
    return df_train.replace(encoding), \
           df_test.replace(encoding), \
           [index[-1]]

In [8]:
def process_f152(df_train, df_test, index):
    f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
    encoding = {'f152': f152_encode}
    return df_train.replace(encoding), \
           df_test.replace(encoding), \
           index

In [9]:
def process_categorical(df_train, df_test):
    cat = df_train.dtypes[df_train.dtypes == type(object)]
    cat_index = list(cat.index)
    
    df_train_res, df_test_res, cat_index = process_f1(df_train, df_test, cat_index)
    df_train_res, df_test_res, cat_index = process_f11(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_f12(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_bool(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_f152(df_train_res, df_test_res, cat_index)
    return df_train_res, df_test_res

In [10]:
from sklearn.impute import KNNImputer

def impute_na(df_train, df_test):
    imputer = KNNImputer(n_neighbors=30, weights='distance')    
    return pd.DataFrame(imputer.fit_transform(df_train), columns=df_train.columns), \
           pd.DataFrame(imputer.fit_transform(df_test), columns=df_test.columns)

In [11]:
def preprocess(df_train, df_test):
    df_train_res, df_test_res = process_categorical(df_train, df_test)
    df_train_res, df_test_res = impute_na(df_train_res, df_test_res)
    return df_train_res, df_test_res

In [12]:
df_train_res, df_test_res = preprocess(df_train, df_test)

In [13]:
df_train_res.head()

,id,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,Ostankinskoe,Poselenie Kokoshkino,Poselenie Rjazanovskoe,Poselenie Klenovskoe,Poselenie Voronovskoe,Severnoe,Vostochnoe,Poselenie Kievskij,Molzhaninovskoe,Poselenie Mihajlovo-Jarcevskoe
0,1.0,69.0,38.00000,7.0,10.000000,1.000000,2001.000000,2.000000,11.000000,2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,55.0,52.21082,2.0,1.000000,4.000000,1632.014324,2.000000,10.000000,1.079469,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,30.0,16.00000,2.0,10.367332,2.471149,1975.601328,1.378480,6.020559,2.609983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,44.0,43.00000,1.0,9.901171,1.512365,1974.037088,1.846787,6.149361,2.171796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,45.0,28.00000,3.0,5.000000,2.000000,1960.000000,2.000000,5.000000,2.489601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
True in df_train_res.isna().values

False

In [15]:
df_test_res.head()

,id,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,Ostankinskoe,Poselenie Kokoshkino,Poselenie Rjazanovskoe,Poselenie Klenovskoe,Poselenie Voronovskoe,Severnoe,Vostochnoe,Poselenie Kievskij,Molzhaninovskoe,Poselenie Mihajlovo-Jarcevskoe
0,1.0,63.0,63.000000,11.0,17.000000,1.000000,1378.518174,2.000000,1.000000,1.858165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,64.0,45.725183,2.0,16.063209,1.365118,312.385500,1.774013,1.211592,1.036679,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,40.0,52.765371,4.0,17.000000,1.000000,1678.864905,1.000000,1.000000,1.150219,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,37.0,1.000000,25.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,127.0,58.000000,11.0,20.000000,1.000000,2006.000000,3.000000,33.000000,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
True in df_test_res.isna().values

False

# Prepare train-test sets

In [17]:
X_train = df_train_res.drop(['id', 'target'], axis=1)
y_train = df_train_res['target']
X_test = df_test_res.drop('id', axis=1)

In [18]:
X_train.shape, y_train.shape, X_test.shape

((24376, 438), (24376,), (6095, 438))

# Train pipeline

In [19]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

def transform(data):
    # normalizer = Normalizer()
    scaler = MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [20]:
from sklearn.metrics import make_scorer

def rmsle(y, y_pred, **kwargs):
    # Allow negative predictions 
    # by incorporating prediction sign
    terms = [np.sign(pred) * np.log(1 + np.sign(pred) * pred) - np.log(1 + targ) for pred, targ in zip(y_pred, y)]
    return np.sqrt(np.sum(np.power(terms, 2.0)) / len(y))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [21]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

def train(model, X, y, score=True, cv=5, tune=False, params=None, n_iter=10):
    X_tr = transform(X)

    if tune:
        if score:
            print(f"Pre-tune CV scores: {cross_val_score(model, X_tr, y, cv=cv, scoring=rmsle_scorer)}")
        
        rscv = RandomizedSearchCV(model, params, n_iter=n_iter, scoring=rmsle_scorer, refit=True, cv=cv, random_state=3927)
        rscv.fit(X_tr, y)
        print(f"Best params: {rscv.best_params_}")
        model_tr = rscv.best_estimator_

        if score:
            print(f"Post-tune CV scores: {cross_val_score(model_tr, X_tr, y, cv=cv, scoring=rmsle_scorer)}")
    else:
        model_tr = model.fit(X_tr, y)
    
        if score:
            print(f"CV scores: {cross_val_score(model_tr, X_tr, y, cv=cv, scoring=rmsle_scorer)}")

    return model_tr

# Test pipeline

In [22]:
def test(model, X):
    X_tr = transform(X)

    pred = model.predict(X_tr)
    data = list(zip(range(1, len(pred) + 1), pred))
    return pd.DataFrame(data, columns=['id', 'prediction'])

# Setup model

## Ridge

In [31]:
from sklearn.linear_model import Ridge
from scipy.stats import loguniform

In [29]:
model = Ridge(random_state=3927)
params = {'alpha': loguniform(0.001, 1000)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=100)
model

Pre-train CV scores: [-0.76793317 -0.80475525 -0.64224988 -0.84887315 -0.66290979]
Best params: {'alpha': 61.80777706558883}
Post-train CV scores: [-0.51919921 -0.52629284 -0.52775206 -0.53538359 -0.54654726]


Ridge(alpha=61.80777706558883, random_state=3927)

In [30]:
model = Ridge(random_state=3927)
params = {'alpha': uniform(30, 60)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=100)
model

Pre-train CV scores: [-0.76793317 -0.80475525 -0.64224988 -0.84887315 -0.66290979]
Best params: {'alpha': 50.54988329108238}
Post-train CV scores: [-0.51624617 -0.52712394 -0.52483883 -0.53329103 -0.54376421]


Ridge(alpha=50.54988329108238, random_state=3927)

## ElasticNet

In [31]:
from sklearn.linear_model import ElasticNet
from scipy.stats import loguniform, uniform

In [32]:
model = ElasticNet(max_iter=500, selection='random', random_state=3927)
params = {'alpha': loguniform(0.01, 100), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-train CV scores: [-0.59307105 -0.59023686 -0.60216332 -0.59788463 -0.61329771]
Best params: {'alpha': 0.034339915870309524, 'l1_ratio': 0.7688000396844558}
Post-train CV scores: [-0.53471431 -0.5386356  -0.54309276 -0.54873848 -0.56099401]


ElasticNet(alpha=0.034339915870309524, l1_ratio=0.7688000396844558,
           max_iter=500, random_state=3927, selection='random')

## DecisionTree

In [33]:
from sklearn.tree import DecisionTreeRegressor

In [34]:
model = DecisionTreeRegressor(random_state=3927)
params = {"max_depth" : [None, 1, 3, 5, 7, 9],
          "min_samples_leaf": [1, 3, 5, 7, 9],
          "min_weight_fraction_leaf": [0.1, 0.2, 0.3, 0.4, 0.5],
          "max_leaf_nodes": [None, 10, 30, 50, 70, 90]}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=20)
model

Pre-train CV scores: [-0.58373169 -0.5816293  -0.57902527 -0.6197102  -0.62401154]
Best params: {'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 3, 'max_leaf_nodes': 70, 'max_depth': None}
Post-train CV scores: [-0.52875864 -0.52792756 -0.53634052 -0.54510319 -0.55090827]


DecisionTreeRegressor(max_leaf_nodes=70, min_samples_leaf=3,
                      min_weight_fraction_leaf=0.1, random_state=3927)

## SGD

In [35]:
from sklearn.linear_model import SGDRegressor
from scipy.stats import loguniform, uniform

In [53]:
model = SGDRegressor(max_iter=5000, penalty='l2', random_state=3927)
params = {'alpha': uniform(0.001, 1000)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.52000473 -0.8025199  -0.52320905 -0.67421686 -0.66528009]
Best params: {'alpha': 375.96342432314395}
Post-tune CV scores: [-0.62219392 -0.61346204 -0.62751864 -0.62138989 -0.63336703]


SGDRegressor(alpha=375.96342432314395, max_iter=5000, random_state=3927)

In [36]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': loguniform(0.001, 1000), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-train CV scores: [-0.52058201 -0.80294035 -0.52343453 -0.67453236 -0.66563684]
Best params: {'alpha': 0.010951283003602989, 'l1_ratio': 0.39457605876595403}
Post-train CV scores: [-0.52893169 -0.53256302 -0.53680113 -0.54323188 -0.55348366]


SGDRegressor(alpha=0.010951283003602989, l1_ratio=0.39457605876595403,
             max_iter=5000, penalty='elasticnet', random_state=3927)

In [37]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': uniform(0.001, 1), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-train CV scores: [-0.52058201 -0.80294035 -0.52343453 -0.67453236 -0.66563684]
Best params: {'alpha': 0.13494980671099222, 'l1_ratio': 0.7688000396844558}
Post-train CV scores: [-0.54447039 -0.54486867 -0.56713527 -0.55965897 -0.57757687]


SGDRegressor(alpha=0.13494980671099222, l1_ratio=0.7688000396844558,
             max_iter=5000, penalty='elasticnet', random_state=3927)

In [38]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': uniform(0.005, 0.05), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-train CV scores: [-0.52058201 -0.80294035 -0.52343453 -0.67453236 -0.66563684]
Best params: {'alpha': 0.01169749033554961, 'l1_ratio': 0.7688000396844558}
Post-train CV scores: [-0.51815211 -0.53837724 -0.53557793 -0.54162234 -0.56373763]


SGDRegressor(alpha=0.01169749033554961, l1_ratio=0.7688000396844558,
             max_iter=5000, penalty='elasticnet', random_state=3927)

## Stacking

In [39]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import StackingRegressor

In [42]:
estimators = [
    ('RIDGE', Ridge(alpha=50, random_state=3927)),
    ('ELNET', ElasticNet(max_iter=500, alpha=0.05, l1_ratio=0.6, selection='random', random_state=3927)),
    ('SGDR', SGDRegressor(max_iter=5000, penalty='elasticnet', alpha=0.01, l1_ratio=0.6, random_state=3927))
]
final = DecisionTreeRegressor(max_leaf_nodes=70, min_samples_leaf=3,
                                    min_weight_fraction_leaf=0.1, random_state=3927)
model = StackingRegressor(estimators, final_estimator=final, n_jobs=-1)

model = train(model, X_train, y_train)
model

CV scores: [-0.52604013 -0.52469718 -0.53491436 -0.54381605 -0.55119607]


StackingRegressor(estimators=[('RIDGE', Ridge(alpha=50, random_state=3927)),
                              ('ELNET',
                               ElasticNet(alpha=0.05, l1_ratio=0.6,
                                          max_iter=500, random_state=3927,
                                          selection='random')),
                              ('SGDR',
                               SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                            max_iter=5000, penalty='elasticnet',
                                            random_state=3927))],
                  final_estimator=DecisionTreeRegressor(max_leaf_nodes=70,
                                                        min_samples_leaf=3,
                                                        min_weight_fraction_leaf=0.1,
                                                        random_state=3927),
                  n_jobs=-1)

## Bagging

In [26]:
from sklearn.ensemble import BaggingRegressor
from scipy.stats import randint

In [29]:
model = BaggingRegressor(n_estimators=5, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=30)
model

Pre-tune CV scores: [-0.45166694 -0.45426409 -0.46059833 -0.4687825  -0.47241517]
Best params: {'max_features': 15, 'max_samples': 53}
Post-tune CV scores: [-0.61141405 -0.61218591 -0.62413512 -0.62759376 -0.62476246]


BaggingRegressor(max_features=15, max_samples=53, n_estimators=5, n_jobs=-1,
                 random_state=3927)

In [33]:
base = Ridge(alpha=50, random_state=3927)
model = BaggingRegressor(base_estimator=base, n_estimators=20, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.51639557 -0.52732489 -0.52465283 -0.53375375 -0.54417277]
Best params: {'max_features': 17, 'max_samples': 93}
Post-tune CV scores: [-0.620283   -0.62007302 -0.62901421 -0.62766516 -0.63624252]


BaggingRegressor(base_estimator=Ridge(alpha=50, random_state=3927),
                 max_features=17, max_samples=93, n_estimators=20, n_jobs=-1,
                 random_state=3927)

In [39]:
base = SGDRegressor(max_iter=5000, alpha=0.01, l1_ratio=0.6, penalty='elasticnet', random_state=3927)
model = BaggingRegressor(base_estimator=base, n_estimators=2, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.52665917 -0.52868404 -0.54358468 -0.54431884 -0.54963534]
Best params: {'max_features': 17, 'max_samples': 93}
Post-tune CV scores: [-0.60621947 -0.59940176 -0.61378721 -0.61133922 -0.61811996]


BaggingRegressor(base_estimator=SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                             max_iter=5000,
                                             penalty='elasticnet',
                                             random_state=3927),
                 max_features=17, max_samples=93, n_estimators=2, n_jobs=-1,
                 random_state=3927)

## Boosting

### AdaBoost

In [40]:
from sklearn.ensemble import AdaBoostRegressor

In [42]:
model = AdaBoostRegressor(n_estimators=10, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.54204225 -0.52067484 -0.54264759 -0.54824418 -0.56108101]


AdaBoostRegressor(loss='square', n_estimators=10, random_state=3927)

In [51]:
base = Ridge(alpha=50, random_state=3927)
model = AdaBoostRegressor(base_estimator=base, n_estimators=2, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.51693623 -0.71379555 -0.52619165 -0.6600755  -0.54584992]


AdaBoostRegressor(base_estimator=Ridge(alpha=50, random_state=3927),
                  loss='square', n_estimators=2, random_state=3927)

In [52]:
base = SGDRegressor(max_iter=5000, alpha=0.01, l1_ratio=0.6, penalty='elasticnet', random_state=3927)
model = AdaBoostRegressor(base_estimator=base, n_estimators=2, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.51972606 -0.65399857 -0.53793218 -0.54556841 -0.5572942 ]


AdaBoostRegressor(base_estimator=SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                              max_iter=5000,
                                              penalty='elasticnet',
                                              random_state=3927),
                  loss='square', n_estimators=2, random_state=3927)

### GradientBoosting

In [54]:
from sklearn.ensemble import GradientBoostingRegressor

In [55]:
model = GradientBoostingRegressor(n_estimators=10, random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.5144895  -0.51580978 -0.52648808 -0.52744631 -0.54062705]


GradientBoostingRegressor(n_estimators=10, random_state=3927)

In [56]:
model = GradientBoostingRegressor(n_estimators=100, random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.45537377 -0.45971252 -0.46682355 -0.4773608  -0.48815675]


GradientBoostingRegressor(random_state=3927)

## Custom

In [75]:
from scipy.optimize import minimize

def loss_function(beta, X, Y):
    return rmsle(Y, X @ beta)

X_train_tr = transform(pd.DataFrame(np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1), columns=list(X_train.columns) + ['intercept']))
beta_init = np.array([1] * X_train_tr.shape[1])
res = minimize(loss_function, beta_init, args=(X_train_tr[:100], y_train[:100]), options={'maxiter': 100, 'disp': True})
print(res.x)

         Current function value: 12.055089
         Iterations: 59
         Function evaluations: 35574
         Gradient evaluations: 121
[-5.89652413e+02 -7.01089037e+02 -9.62186457e+02  3.33754164e+02
  3.42381794e+02  1.30231496e+03  1.79167827e+03  1.43884259e+03
 -7.49728366e+01  5.79214141e+01  5.28944642e+02  1.83410402e+02
  4.22472647e+02 -4.17908865e+03 -4.98516620e+02 -1.61690452e+00
  1.78767081e+03 -1.72826181e+02 -1.18831036e+02  8.27094063e+02
  1.01015129e+01 -1.75484736e+02  5.85650009e+02 -1.86386872e+02
 -2.81608946e+02 -4.88085043e+02  2.36373631e+02 -6.46890542e+02
 -5.44585001e+02 -7.94931163e+02 -1.52572017e+03 -1.24195282e+03
 -6.74089905e+02 -1.37164174e+03 -3.09696457e+02 -3.64522345e+02
  1.55807784e+03  7.06889666e+02 -9.76006292e+02 -8.33134505e+02
 -1.16224576e+03 -1.70815689e+03  2.13133270e+03  2.09622797e+03
  2.16019188e+03 -1.45156673e+02 -1.12336625e+02 -1.79945344e+02
 -3.95278263e+01  1.07097226e+02 -1.88264431e+02  1.96235380e+02
  3.29597084e+02

# Test

In [57]:
df_pred = test(model, X_test)
df_pred.head()

,id,prediction
0,1,1.703304e+07
1,2,5.037466e+06
2,3,1.396499e+07
3,4,1.744371e+07
4,5,5.272219e+07


# Validate

In [58]:
df_pred['prediction'][df_pred['prediction'] <= 100000]

Series([], Name: prediction, dtype: float64)

# Export

In [59]:
df_pred.to_csv("output/submission.csv", index=False)